In [94]:
import pandas as pd
path = '../data/apartments_modeling_data.csv'

data = pd.read_csv(path)


In [95]:
data

,rooms,bedrooms,bath,no_proximity_newdev,BoroID,Boro,rent_price_cleaned,amenities_fl,pre_flag
0,2,0,1.0,True,3,BROOKLYN,1900.000000,1,1
1,3,1,1.0,True,1,MANHATTAN,2210.000000,1,1
2,4,2,2.5,True,3,BROOKLYN,4000.000000,1,1
3,7,3,1.5,True,4,QUEENS,2000.000000,0,1
4,2,0,1.0,True,1,MANHATTAN,1950.000000,0,1
...,...,...,...,...,...,...,...,...,...
519,4,1,1.0,True,4,QUEENS,1725.000000,0,0
520,4,2,2.0,False,3,BROOKLYN,3295.000000,1,0
521,4,3,1.0,True,3,BROOKLYN,2566.666667,0,0
522,5,3,1.0,True,3,BROOKLYN,2575.000000,0,0


In [96]:
X = data[['rooms','bedrooms','bath','no_proximity_newdev','BoroID','amenities_fl','pre_flag']]


In [97]:
# Introducing a new column of binary values for boolean 'new_proximity_newdev'
new_col = []
for i in X['no_proximity_newdev']:
    if i  == True:
        new_col.append(1)
    else:
        new_col.append(0)
X = X.assign(no_proximity_newdev_bin=new_col) # Name of new column - no_proximity_newdev_bin

In [98]:
#Introducing the column for the new variable 'no_proximity_newdev_bin*pre_flag'
new_prod_col = []
for index, row in X.iterrows():
    new_prod_col.append(row['no_proximity_newdev_bin']* row['pre_flag'])
X = X.assign(product_column=new_prod_col) # Column name - 'product_column'
del X['no_proximity_newdev']

In [99]:
X

,rooms,bedrooms,bath,BoroID,amenities_fl,pre_flag,no_proximity_newdev_bin,product_column
0,2,0,1.0,3,1,1,1,1
1,3,1,1.0,1,1,1,1,1
2,4,2,2.5,3,1,1,1,1
3,7,3,1.5,4,0,1,1,1
4,2,0,1.0,1,0,1,1,1
...,...,...,...,...,...,...,...,...
519,4,1,1.0,4,0,0,1,0
520,4,2,2.0,3,1,0,0,0
521,4,3,1.0,3,0,0,1,0
522,5,3,1.0,3,0,0,1,0


### Linear Regression with Ordinary Least Squares(OLS)

In [100]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

y = data['rent_price_cleaned'].values
X_new = X 


# Get one hot encoding of column "BoroID" 
one_hot = pd.get_dummies(X_new['BoroID'])
X_new = X_new.drop('BoroID',axis = 1)
X_new = X_new.join(one_hot)
X_new

,rooms,bedrooms,bath,amenities_fl,pre_flag,no_proximity_newdev_bin,product_column,1,2,3,4
0,2,0,1.0,1,1,1,1,0,0,1,0
1,3,1,1.0,1,1,1,1,1,0,0,0
2,4,2,2.5,1,1,1,1,0,0,1,0
3,7,3,1.5,0,1,1,1,0,0,0,1
4,2,0,1.0,0,1,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
519,4,1,1.0,0,0,1,0,0,0,0,1
520,4,2,2.0,1,0,0,0,0,0,1,0
521,4,3,1.0,0,0,1,0,0,0,1,0
522,5,3,1.0,0,0,1,0,0,0,1,0


In [101]:

X_mat = np.array(X_new, dtype=float)

model = sm.OLS(y, X_mat)
results = model.fit()
print(results.summary())

#############
#For reference
# x1 - rooms
# x2 - bedrooms
# x3 - bath
# x4 - BoroID(one hot encoded to Manhattan)
# x4 - amenities_fl
# x5 - pre_flag
# x6 - no_proximity_newdev_bin
# x7 - product_column(coefficient needs to be observed) <----- IMPORTANT
# x8 - 1 if Manhattan, 0 otherwise
# x9 - 1 if Bronx, 0 otherwise
# x10 - 1 if Brooklyn, 0 otherwise
# x11 - 1 if Queens, 0 otherwise
#############

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.413
Model:                            OLS   Adj. R-squared:                  0.401
Method:                 Least Squares   F-statistic:                     36.07
Date:                Tue, 30 Mar 2021   Prob (F-statistic):           2.68e-53
Time:                        20:28:38   Log-Likelihood:                -4290.9
No. Observations:                 524   AIC:                             8604.
Df Residuals:                     513   BIC:                             8651.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            27.1365     57.547      0.472      0.6